# Part 1: analyze demos

In [ ]:
import os, sys
import sqlite3
import pandas as pd

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW

db_name = 'rtcw_db.db'
demos_folder = 'demos'

cwd = os.getcwd()
root_path = os.path.join(cwd, '../')
demos_path = os.path.join(cwd, '../', demos_folder)

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

In [3]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [4]:
obituary_df = db.get_table_as_df(table_name= 'obituary')

obituary table fetched in 0.51 seconds


In [5]:
#example, get all 5 man panzerfausts
df_spree = RTCW.get_kill_sprees(obituary_df, 
                                maxtime_secs = 2, 
                                weapon_filter = ['panzerfaust'], 
                                minspree = 5, 
                                verbose = True)

scanned 100 demos of 928 demos in total
scanned 200 demos of 928 demos in total
scanned 300 demos of 928 demos in total
scanned 400 demos of 928 demos in total
scanned 500 demos of 928 demos in total
scanned 600 demos of 928 demos in total
scanned 700 demos of 928 demos in total
scanned 800 demos of 928 demos in total
scanned 900 demos of 928 demos in total


In [6]:
df_spree

,attacker,end,md5,spreecount,start
0,4,18869722,812d4548ed3716988ce3c615653b1454,5,18869722
1,8,54531750,b23cdd6102c08e440380e841fbd6fbba,5,54531750
2,1,46549650,97004b41f3cbde43dd8b47c454439ee1,5,46549650
3,11,18834700,1723b976917e6f121e0fe8e659714801,5,18834700
4,12,51868850,ad7010534bceab5bb417015305d3d1f2,5,51868850
5,11,742567063,b24f32770cc0c602d4ec50e4742c8d4a,5,742567063
6,6,5321200,d31c87d84c2dda10302b31b5da94993b,5,5321200


# Part 2: cut demos

In [38]:
exe_name = 'Anders.Gaming.LibTech3.exe'
demos_folder = 'demos'
root_path = 'C:\\Users\\JelleGrammens\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path
path = os.getcwd()
exe_path = os.path.join(path, '../', exe_name)

In [47]:
def locate_demo_path(demos_dct, spree, root_path):
    '''
    helper function that outputs the path to the demo that we want to cut
    '''
    for k in demos_dct:
        if spree.md5 in demos_dct[k][0]:
            match_folder = k
            demo_loc = [i for i in range(len(demos_dct[k][0])) if demos_dct[k][0][i]==spree.md5]
            demo_name = demos_dct[k][1][demo_loc[0]]
            
            return match_folder, demo_name

In [48]:
def cutter_exe_cmd(root_path, match_folder, demo_name, start_time, end_time, 
                   demo_folder_name = 'demos', output_folder = 'output_spree_demos', cut_type = 1):
    '''
    helper function to create string with demo_path and parameters to input in anders libtech 3 api
    parameters: 
    - demo_path: full path to demo
    - parameters_dct: a dictionary with all the parameters necessary
    '''
    demo_path = os.path.join(root_path, demo_folder_name, match_folder, demo_name)
    output_path = os.path.join(root_path, output_folder, demo_name)
    
    s = 'cut ' + demo_path + ' '
    s += output_path + ' '
    s += str(start_time) + ' '
    s += str(end_time) + ' '
    s += str(cut_type) + ' 0' # plus the zero at the end or the api will crash

    return s

In [51]:
def cut_demos(root_path, demos_dct, df_spree, exe_name, offset_start = 5000, offset_end = 5000):

    for row in range(len(df_spree)):

        spree = df_spree.loc[row]
        match_folder, demo_name = locate_demo_path(demos_dct, spree, root_path)

        start_time = spree.start - offset_start
        end_time = spree.end + offset_end

        parameters = cutter_exe_cmd(root_path, match_folder, demo_name, start_time, end_time, 
                                    demo_folder_name = 'demos', output_folder = 'output_spree_demos', cut_type = 1)

        os.system(exe_path + ' ' + parameters)

In [52]:
cut_demos(root_path, demos_dct, df_spree, exe_name)